# Setup

In [21]:
!pip install transformers
!pip install -U adapter-transformers
!pip install datasets
!pip install pytorch_lightning
!pip install sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# import transformers
# import datasets

from transformers import (
    # AdamW, 
    T5Model, 
    T5ForConditionalGeneration, 
    T5AdapterModel, 
    T5Tokenizer, 
    get_linear_schedule_with_warmup,
    TrainingArguments, 
    AdapterTrainer,
    Trainer
)

from dataprovider.DataProvider import DatasetProvider

# Load Data

In [24]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import create_optimizer, AdamWeightDecay
from transformers import TFAutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

In [25]:
df_test = pd.read_csv('data/COMBINED/test.tsv', sep = '\t', names=['in', 'expected'])
df_test

,in,expected
0,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...
1,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...
4,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...
...,...,...
5172,Twice Sparrow sold the island twice to Thomas ...,Sparrow twice sold the island to Thomas Polloc...
5173,The name in Tupi means `` insensitive stone ''...,"The name in Tupi means '' hard stone `` , '' i..."
5174,"The company has branches in Tokyo , based in t...",The company has branches in Tokyo based in Sai...
5175,The modern coat of arms of Bavaria was designe...,The modern coat of arms of Bavaria was designe...


In [26]:
df_train = pd.read_csv('data/COMBINED/train.tsv', sep = '\t', names=['in', 'expected'])
df_train

,in,expected
0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi..."
1,"Referring to him as only ""the witness"", Amrozi...","Amrozi accused his brother, whom he called ""th..."
2,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
3,Yucaipa bought Dominick's in 1995 for $693 mil...,Yucaipa owned Dominick's before selling the ch...
4,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an..."
...,...,...
58613,"Tommy Connolly , who plays Rory Jennings , pla...","Tommy Connolly , who plays Rory Jennings , pla..."
58614,"Monroe Meadows , in Yosemite valley near Brida...","Monroe Meadows , in Yosemite Valley near Brida..."
58615,"Monroe Meadows , in Yosemite Valley near Brida...","Monroe Meadows , in Yosemite valley near Brida..."
58616,In 2014 the site launched iOS and Android appl...,In 2014 launched the site iOS and Android - ap...


In [27]:
###
dataset_test = Dataset.from_pandas(df_test)
dataset_train = Dataset.from_pandas(df_train)

###
init_weights = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(init_weights)

def preprocess_function(examples):
    prefix = 'paraphrase: '
    inputs = [prefix + doc for doc in examples["in"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["expected"], max_length=512, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_test = dataset_test.map(preprocess_function, batched=True)
tokenized_train = dataset_train.map(preprocess_function, batched=True)

###
num_train = 30000
num_test = 1500
tokenized_train_small = tokenized_train.shuffle(seed=42).select(range(100))
tokenized_test_small = tokenized_test.shuffle(seed=42).select(range(10))

###
tokenized_train_small.set_format(type="torch", columns=["attention_mask", "input_ids", "labels"],)
tokenized_test_small.set_format(type="torch", columns=["attention_mask", "input_ids", "labels"], )

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /Users/yichen/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "leng

In [28]:
type(tokenized_train_small)

datasets.arrow_dataset.Dataset

# Adapter Training

## Setup Model

In [29]:
adapter_model = T5AdapterModel.from_pretrained("t5-small")
adapter_model.add_seq2seq_lm_head("lm_head")
adapter_model.add_adapter("paraphrase1")
adapter_model.train_adapter("paraphrase1")

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /Users/yichen/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
  

## Setup Training

In [30]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-4,
    weight_decay=0.01,
    num_train_epochs=3,
    remove_unused_columns=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=adapter_model)

trainer = AdapterTrainer(
    model=adapter_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_train_small,
    eval_dataset=tokenized_test_small,
    data_collator=data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Train

In [32]:
trainer.train()

***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
 33%|███▎      | 13/39 [00:16<00:27,  1.05s/it]***** Running Evaluation *****
  Num examples = 10
  Batch size = 8



                                                     

 33%|███▎      | 13/39 [00:16<00:27,  1.05s/it]



{'eval_loss': 1.1636545658111572, 'eval_runtime': 0.8603, 'eval_samples_per_second': 11.624, 'eval_steps_per_second': 2.325, 'epoch': 1.0}


 67%|██████▋   | 26/39 [00:33<00:14,  1.12s/it]***** Running Evaluation *****
  Num examples = 10
  Batch size = 8



                                                  

                                               
 67%|██████▋   | 26/39 [00:33<00:14,  1.12s/it]


{'eval_loss': 1.1636545658111572, 'eval_runtime': 0.8448, 'eval_samples_per_second': 11.838, 'eval_steps_per_second': 2.368, 'epoch': 2.0}


100%|██████████| 39/39 [00:50<00:00,  1.10s/it]***** Running Evaluation *****
  Num examples = 10
  Batch size = 8



                                                  

                                               
100%|██████████| 39/39 [00:51<00:00,  1.10s/it]


Training completed. Do not forget to share your model on huggingface.co/models =)



100%|██████████| 39/39 [00:51<00:00,  1.32s/it]      

{'eval_loss': 1.1636545658111572, 'eval_runtime': 0.8876, 'eval_samples_per_second': 11.266, 'eval_steps_per_second': 2.253, 'epoch': 3.0}
{'train_runtime': 51.4115, 'train_samples_per_second': 5.835, 'train_steps_per_second': 0.759, 'train_loss': 0.9082233722393329, 'epoch': 3.0}


TrainOutput(global_step=39, training_loss=0.9082233722393329, metrics={'train_runtime': 51.4115, 'train_samples_per_second': 5.835, 'train_steps_per_second': 0.759, 'train_loss': 0.9082233722393329, 'epoch': 3.0})

# Save

In [34]:
adapter_model.save_adapter("adapter_paraphrase", "paraphrase1")

Configuration saved in adapter_paraphrase/adapter_config.json
Module weights saved in adapter_paraphrase/pytorch_adapter.bin


In [ ]:
adapter_model.save_pretrained('results/checkpoint/t5adapter-save')

In [35]:
adapter_model.save_adapter("results/checkpoint/adapter_paraphrase1", "paraphrase1")

FileNotFoundError: [Errno 2] No such file or directory: 'results/checkpoint/adapter_paraphrase1'